In [2]:
import numpy as np
import hdf5plugin
import h5py
#import matplotlib.pyplot as plt
import blosc2
import blosc
import lz4
import lz4.frame
import bitshuffle

In [23]:
import ctypes
import ctypes.util

In [26]:
ctypes.util.find_library('lz4')

'/home/beams/SKANDEL/miniforge3/envs/edge2/lib/liblz4.so.1'

In [3]:
data = np.load('diff_scan_810_128x128.npy')

In [32]:
data_padded = np.pad(data, [[0,0], [192, 192],[192,192]])
data0 = data_padded[:8, ...]

In [5]:
data0.shape

(1, 512, 512)

In [28]:
import tifffile

In [33]:
%%timeit
np.save('diff_scan_810_512x512.npy', data0)

18.1 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
with h5py.File('diff_scan_810_512x512.hdf5', 'w') as f:
    dct = dict(hdf5plugin.LZ4())
    print(dct)
    f.create_dataset('data', data=data0, **dct, chunks=(1, 512, 512))
    print(f['data']._filters)
    test_dat = f['data'].id.read_direct_chunk((0,0,0))
    
    
    
    

{'compression': 32004, 'compression_opts': (0,)}
{'32004': (0,)}


In [9]:
uts = np.frombuffer(test_dat[1], dtype='uint8')

In [19]:
d_context = lz4.frame.create_decompression_context()

In [21]:
lz4.frame.decompress_chunk(d_context, uts)

RuntimeError: LZ4F_decompress failed with code: ERROR_frameType_unknown

In [23]:
import lz4
import lz4.frame

In [14]:
bitshuffle.decompress_lz4(uts, shape=data0.shape)

TypeError: decompress_lz4() takes at least 3 positional arguments (2 given)

In [16]:
decomp = blosc2.decompress2(uts)
d2 = np.frombuffer(decomp, dtype='float32')

ValueError: Error while decompressing, check the src data and/or the dparams

In [19]:
blosc2.decompress??

Signature: blosc2.decompress(src, dst=None, as_bytearray=False)
Source:   
def decompress(src, dst=None, as_bytearray=False):
    """Decompresses a bytes-like compressed object.

    Parameters
    ----------
    src : bytes-like object
        The data to be decompressed.  Must be a bytes-like object
        that supports the Python Buffer Protocol, like bytes, bytearray,
        memoryview, or
        `numpy.ndarray <https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html>`_.
    dst : NumPy object or bytearray
        The destination NumPy object or bytearray to fill,
        the length of which must be greater than 0.
        The user must make sure
        that it has enough capacity for hosting the decompressed data.
        Default is None, meaning that a new `bytes` or `bytearray` object
        is created, filled and returned.
    as_bytearray : bool (optional)
        If this flag is True then the return type will be a bytearray object
        instead of a bytes o

In [100]:
d2.shape

(262144,)

In [83]:
dat1 = data0.tobytes()

In [ ]:
dat2 = dat0.

In [80]:
dat2 = bytearray(data0.ravel())

In [81]:
dat2[:10], dat1[:10]

(bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [82]:
dat3 = dat2.decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 393994: invalid start byte

In [78]:
dat2[:10].decode()

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [90]:
dat2 = np.frombuffer(dat1, dtype='uint8')

In [91]:
dat3 = np.frombuffer(dat2, dtype='float32')

In [92]:
len(dat3)

262144

In [262]:
decompressed = blosc.decompress(bytearray(test_dat[1]))

In [265]:
dec = np.frombuffer(decompressed, dtype='float32').reshape(1, 512, 512)

In [53]:
f1 = h5py.File('diff_scan_810_512x512.hdf5', 'r')

In [55]:
dataset = f1['data']

In [56]:
dataset._filters

{'32001': (2, 2, 4, 126976, 5, 1, 1)}

In [58]:
dataset.id.read_direct_chunk((1,0,0))

OSError: Can't read unprocessed chunk data (offset doesn't fall on chunks's boundary)

In [62]:
f1.close()

In [23]:
data.max()

48413.125

In [21]:
data.shape, data_padded.shape

((963, 128, 128), (963, 512, 512))